# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-23'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2513,2022-11-18,"$16,687.91","$16,947.06","$16,564.61","$16,697.78","$16,697.78",26862218609
2514,2022-11-19,"$16,696.22","$16,797.88","$16,570.41","$16,711.55","$16,711.55",16106223492
2515,2022-11-20,"$16,712.92","$16,746.78","$16,248.69","$16,291.83","$16,291.83",21313378652
2516,2022-11-21,"$16,291.22","$16,291.22","$15,599.05","$15,787.28","$15,787.28",37429485518
2517,2022-11-22,"$15,782.30","$16,253.05","$15,656.61","$16,189.77","$16,189.77",30726828760


In [ ]:
# Validamos que no hay datos vacíos


In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[["Date","Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds",
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns={'Date':'ds','Open':'y'})


In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
2513,2022-11-18,"$16,687.91"
2514,2022-11-19,"$16,696.22"
2515,2022-11-20,"$16,712.92"
2516,2022-11-21,"$16,291.22"
2517,2022-11-22,"$15,782.30"


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Bitcoin"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode="multiplicative")

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2878,2023-11-18
2879,2023-11-19
2880,2023-11-20
2881,2023-11-21
2882,2023-11-22


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
2878,2023-11-18
2879,2023-11-19
2880,2023-11-20
2881,2023-11-21


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2878,2023-11-18,"$-15,375.99","$-43,657.78","$11,254.26"
2879,2023-11-19,"$-15,456.05","$-45,037.96","$11,324.63"
2880,2023-11-20,"$-15,529.98","$-44,914.91","$10,422.84"
2881,2023-11-21,"$-15,592.51","$-45,495.95","$10,033.58"
2882,2023-11-22,"$-15,629.65","$-45,816.96","$11,323.77"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)